---
title: "Can you really 'say what you mean'?"
date: "12/01/2024"
date-modified: last-modified
description-meta: "A look at the impact of structured outputs on the performance of proprietary LLMs."
toc: true
toc-depth: 3
lightbox: true
fig-cap-location: margin
categories:
  - llm
  - openai
  - pydantic
  - python
author:
  - name: Dylan Castillo
    url: https://dylancastillo.co
    affiliation: Iwana Labs
    affiliation-url: https://iwanalabs.com
citation: true
comments:
  utterances:
    repo: dylanjcastillo/blog_comments
    theme: dark-blue
    issue-term: pathname
---

In [261]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [262]:
# | output: false
# | echo: false

import nest_asyncio

nest_asyncio.apply()

In [263]:
# | output: false

import asyncio
import json
import os
from asyncio import Semaphore
from enum import Enum
from textwrap import dedent
from typing import Callable, List, Literal

import numpy as np
import pandas as pd
from datasets import load_dataset
from dotenv import load_dotenv
from langsmith import traceable
from langsmith.wrappers import wrap_openai
from openai import AsyncOpenAI
from openai.types.chat import ChatCompletion
from pydantic import BaseModel, ConfigDict, Field, ValidationError
from scipy import stats

np.random.seed(42)

load_dotenv()

MODEL_NAME = "gpt-4o-mini"

if MODEL_NAME.startswith("gemini"):
    client = wrap_openai(
        AsyncOpenAI(
            api_key=os.getenv("GEMINI_API_KEY"),
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        )
    )
else:
    client = wrap_openai(AsyncOpenAI())

In [264]:
class PromptType(Enum):
    WITHOUT_STRUCTURED_OUTPUT = "without_so"
    WITH_TOOL_CALLS = "with_so_tool_calls"
    WITH_JSON_MODE = "with_so_json_mode"
    WITH_STRICT_TOOL_CALLS = "with_so_strict_tool_calls"


class ClientConfig(BaseModel):
    name: str
    use_response_model: bool
    col_name: str
    score_col_name: str


CONFIGS = [
    ClientConfig(
        name=PromptType.WITHOUT_STRUCTURED_OUTPUT.value,
        use_response_model=False,
        col_name=f"response_{PromptType.WITHOUT_STRUCTURED_OUTPUT.value}",
        score_col_name=f"score_{PromptType.WITHOUT_STRUCTURED_OUTPUT.value}",
    ),
    ClientConfig(
        name=PromptType.WITH_STRICT_TOOL_CALLS.value,
        use_response_model=True,
        col_name=f"response_{PromptType.WITH_STRICT_TOOL_CALLS.value}",
        score_col_name=f"score_{PromptType.WITH_STRICT_TOOL_CALLS.value}",
    ),
]

if not MODEL_NAME.startswith("gemini"):
    CONFIGS.extend(
        [
            ClientConfig(
                name=PromptType.WITH_TOOL_CALLS.value,
                use_response_model=True,
                col_name=f"response_{PromptType.WITH_TOOL_CALLS.value}",
                score_col_name=f"score_{PromptType.WITH_TOOL_CALLS.value}",
            ),
            ClientConfig(
                name=PromptType.WITH_JSON_MODE.value,
                use_response_model=True,
                col_name=f"response_{PromptType.WITH_JSON_MODE.value}",
                score_col_name=f"score_{PromptType.WITH_JSON_MODE.value}",
            ),
        ]
    )


In [274]:
class LLMEvaluator:
    def __init__(
        self,
        configs: List[ClientConfig],
        create_prompt_fn: Callable,
        parse_response_fn: Callable,
        response_model: BaseModel,
        concurrency: int = 100,
    ):
        self.configs = configs
        self.create_prompt_fn = create_prompt_fn
        self.parse_response_fn = parse_response_fn
        self.response_model = response_model
        self.concurrency = concurrency

    def _create_tool_call_schema(
        self,
        strict: bool = False,
    ) -> dict:
        model_schema = self.response_model.model_json_schema()
        if strict:
            return {
                "type": "json_schema",
                "json_schema": {
                    "name": model_schema["title"],
                    "schema": model_schema,
                    "strict": True,
                },
            }
        else:
            return {
                "type": "function",
                "function": {
                    "name": model_schema["title"],
                    "description": f"Correctly extracted `{model_schema['title']}` with all the required parameters with correct types",
                    "parameters": model_schema,
                },
            }

    @traceable(run_type="prompt")
    def create_prompt(
        self,
        question: str,
        prompt_type: str,
    ) -> List[dict]:
        return self.create_prompt_fn(
            question=question,
            prompt_type=prompt_type,
            response_model=self.response_model,
        )

    @traceable(run_type="parser")
    def parse_response(
        self,
        response: ChatCompletion,
        prompt_type: str,
    ) -> str | int:
        return self.parse_response_fn(response, prompt_type)

    @traceable(run_type="llm")
    async def call_llm(
        self,
        config: ClientConfig,
        question: str,
    ) -> ChatCompletion:
        params = {
            "messages": self.create_prompt(question=question, prompt_type=config.name),
            "model": MODEL_NAME,
            "timeout": 120,
        }

        prompt_type_configs = {
            PromptType.WITH_JSON_MODE.value: {
                "response_format": {"type": "json_object"}
            },
            PromptType.WITH_TOOL_CALLS.value: {
                "tools": [self._create_tool_call_schema(strict=False)]
            },
            PromptType.WITH_STRICT_TOOL_CALLS.value: {
                "response_format": self._create_tool_call_schema(strict=True)
            },
        }

        if config.name in prompt_type_configs:
            params.update(prompt_type_configs[config.name])

        completion = await client.chat.completions.create(**params)

        if config.name == PromptType.WITH_TOOL_CALLS.value:
            response_content = (
                completion.choices[0].message.tool_calls[0].function.arguments
            )
        else:
            response_content = completion.choices[0].message.content

        return response_content

    @traceable(run_type="chain")
    async def process_question(
        self,
        question: str,
        config: ClientConfig,
        semaphore: Semaphore,
        max_attempts: int = 3,
    ) -> str | int | None:
        async with semaphore:
            for _ in range(max_attempts):
                try:
                    answer = await self.call_llm(
                        config=config,
                        question=question,
                    )
                    parsed_answer = self.parse_response(answer, config.name)
                    return parsed_answer
                except Exception as e:
                    # print(
                    #     f"{config.name}: Error processing question {question}: {e}. Will try again."
                    # )
                    await asyncio.sleep(1)
                    continue
            print(
                f"{config.name}: Failed to process question {question}, after 3 attempts. Set answer to null."
            )
        return None

    @traceable(run_type="chain")
    async def process_questions(
        self,
        run_name: str,
        questions: List[dict],
        config: ClientConfig,
    ) -> List[str | int | None]:
        semaphore = Semaphore(self.concurrency)
        tasks = [
            self.process_question(
                question=question["question"],
                config=config,
                semaphore=semaphore,
            )
            for question in questions
        ]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        return results

    def generate_outputs(self, questions: List[dict]) -> pd.DataFrame:
        df = pd.DataFrame(
            {
                "id": [i for i in range(len(questions))],
                "question": [question["question"] for question in questions],
                "answer": [question["answer"] for question in questions],
            }
        )
        for config in self.configs:
            responses = asyncio.run(
                self.process_questions(
                    run_name=config.name,
                    questions=questions,
                    config=config,
                )
            )
            df[config.col_name] = responses
        return df

    def evaluate_outputs(self, df: pd.DataFrame) -> pd.DataFrame:
        df_copy = df.copy()
        for config in self.configs:
            df_copy[config.score_col_name] = (
                df_copy["answer"] == df_copy[config.col_name]
            ) * 1
        return df_copy

    def calculate_confidence_intervals(
        self, df: pd.DataFrame, conf_level: float = 0.95
    ) -> None:
        print(
            f"Calculating confidence intervals ({conf_level}) with {len(df)} observations:"
        )
        for config in self.configs:
            score_col = config.score_col_name
            scores = df[score_col]

            if len(scores) == 0:
                print(f"No scores available for {score_col}")
                continue

            mean_score = scores.mean()
            se_score = scores.std() / np.sqrt(len(scores))

            z_score = stats.norm.ppf((1 + conf_level) / 2)
            margin_error = z_score * se_score
            ci = [
                max(0.0, mean_score - margin_error),
                min(1.0, mean_score + margin_error)
            ]
            print(
                f"{score_col} - Mean: {mean_score * 100:.2f}% CI: {ci[0] * 100:.2f}% - {ci[1] * 100:.2f}%"
            )
        print()

    def run_paired_t_test(self, df: pd.DataFrame) -> None:
        scores = {}

        for config in self.configs:
            score_col = config.score_col_name
            scores[score_col] = df[score_col] * 1

        for score_col_1, score_col_2 in [
            ("score_without_so", "score_with_so_tool_calls"),
            ("score_without_so", "score_with_so_json_mode"),
            ("score_without_so", "score_with_so_strict_tool_calls"),
        ]:
            if score_col_1 in scores and score_col_2 in scores:
                t_stat, p_value = stats.ttest_rel(
                    scores[score_col_1], scores[score_col_2]
                )
                print(f"{score_col_1} vs {score_col_2}")
                print(f"t-statistic: {t_stat}, p-value: {p_value}")

## GSM8K

In [266]:
class ResponseGSM8K(BaseModel):
    model_config = ConfigDict(extra="forbid", title="Response")
    reasoning: str = Field(description="step by step reasoning about the answer")
    answer: int = Field(description="final answer")


def create_prompt_gsm8k(
    question, prompt_type: str, response_model: ResponseGSM8K | None = None
):
    if prompt_type in [
        PromptType.WITH_JSON_MODE.value,
        PromptType.WITH_TOOL_CALLS.value,
        PromptType.WITH_STRICT_TOOL_CALLS.value,
    ]:
        system_prompt = (
            dedent("""
        You are an expert in solving grade school math tasks. You will be presented with a grade-school math word problem and be asked to solve it.

        You will always respond with a JSON object matching the following JSON schema:
        """)
            + json.dumps(response_model.model_json_schema(), indent=2)
            + """\n\nFirst, provide your step by step reasoning in the "reasoning" field. Then, in the "answer" field, provide an integer that corresponds to the correct answer to the question. Don't include any other text in the "answer" field."""
        )
    # elif prompt_type in [
    #     PromptType.WITH_TOOL_CALLS.value,
    #     PromptType.WITH_STRICT_TOOL_CALLS.value,
    # ]:
    #     system_prompt = dedent("""
    #     You are an expert in solving grade school math tasks. You will be presented with a grade-school math word problem and be asked to solve it.
        
    #     You will always respond with a function call with two fields: "reasoning" and "answer".
        
    #     First, provide your step by step reasoning in the "reasoning" field. Then, in the "answer" field, provide an integer that corresponds to the correct answer to the question. Don't include any other text in the "answer" field.
    #     """)
    else:
        system_prompt = dedent("""
        You are an expert in solving grade school math tasks. You will be presented with a grade-school math word problem and be asked to solve it.
        
        You will always respond in the following format:
        
        <str, step by step reasoning about the answer>
        ANSWER: <int, final answer>
        
        First, provide your step by step reasoning. Then, in ANSWER, provide an integer that corresponds to the correct answer to the question. Don't include any other text in ANSWER.
        """)

    example_question = [
        "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
        "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
        "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
    ]
    example_explanation = [
        "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.",
        "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.",
        "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.",
    ]
    example_answer = [6, 5, 39]

    # for example_q, example_reason, example_ans in zip(example_question, example_explanation, example_answer):
    #     messages.append(
    #         {
    #             "role": "user",
    #             "content": f"Question: {example_q}",
    #         }
    #     )
    #     if prompt_type in [
    #         PromptType.WITH_JSON_MODE.value,
    #         PromptType.WITH_TOOL_CALLS.value,
    #         PromptType.WITH_STRICT_TOOL_CALLS.value,
    #     ]:
    #         response = f'{{"reasoning": "{example_reason}", "answer": "{example_ans}"}}'
    #     else:
    #         response = f"{example_reason}\nANSWER: {example_ans}"

    #     messages.append({"role": "assistant", "content": response})

    # messages.append({"role": "user", "content": f"Question: {question}"})

    system_prompt += "\nExamples:\n"
    for i, (example_q, example_reason, example_ans) in enumerate(
        zip(example_question, example_explanation, example_answer)
    ):
        system_prompt += f"\n\n**{i+1}**\nQuestion: {example_q}"
        if prompt_type in [
            PromptType.WITH_JSON_MODE.value,
            PromptType.WITH_TOOL_CALLS.value,
            PromptType.WITH_STRICT_TOOL_CALLS.value,
        ]:
            response = f'{{"reasoning": "{example_reason}", "answer": {example_ans}}}'
        else:
            response = f"{example_reason}\nANSWER: {example_ans}"
        system_prompt += f"\nAssistant Response:\n{response}"

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"Question: {question}",
        },
    ]

    return messages


def parse_response_gsm8k(response: str, prompt_type: str) -> int | None:
    if prompt_type in [
        PromptType.WITH_JSON_MODE.value,
        PromptType.WITH_TOOL_CALLS.value,
        PromptType.WITH_STRICT_TOOL_CALLS.value,
    ]:
        return ResponseGSM8K.model_validate_json(response).answer
    else:
        cleaned_response = (
            response.split("\nANSWER:")[1].replace(",", "").rstrip(".").strip()
        )
        return int(cleaned_response)

In [268]:
dataset = load_dataset("gsm8k", "main")
evals = [
    {
        "question": d["question"],
        "answer": int(d["answer"].split("#### ")[1].replace(",", "").strip()),
    }
    for d in dataset["test"]
][:10]

evaluator = LLMEvaluator(
    configs=CONFIGS,
    create_prompt_fn=create_prompt_gsm8k,
    parse_response_fn=parse_response_gsm8k,
    response_model=ResponseGSM8K,
)

df = evaluator.generate_outputs(evals)
df_results = evaluator.evaluate_outputs(df)

In [269]:
evaluator.calculate_confidence_intervals(df_results)
evaluator.run_paired_t_test(df_results)

Calculating confidence intervals (0.95) with 10 observations:
score_without_so - Mean: 100.00% CI: 100.00% - 100.00%
score_with_so_strict_tool_calls - Mean: 100.00% CI: 100.00% - 100.00%
score_with_so_tool_calls - Mean: 100.00% CI: 100.00% - 100.00%
score_with_so_json_mode - Mean: 100.00% CI: 100.00% - 100.00%

score_without_so vs score_with_so_tool_calls
t-statistic: nan, p-value: nan
score_without_so vs score_with_so_json_mode
t-statistic: nan, p-value: nan
score_without_so vs score_with_so_strict_tool_calls
t-statistic: nan, p-value: nan


## Last Letter

In [270]:
class ResponseLastLetter(BaseModel):
    model_config = ConfigDict(extra="forbid", title="Response")
    reasoning: str = Field(description="step by step reasoning about the answer")
    answer: str = Field(description="lowercase concatenated last letters")


def create_prompt_last_letter(
    question, prompt_type: str, response_model: ResponseLastLetter | None = None
):
    if prompt_type in [
        PromptType.WITH_JSON_MODE.value,
        PromptType.WITH_TOOL_CALLS.value,
        PromptType.WITH_STRICT_TOOL_CALLS.value,
    ]:
        system_prompt = (
            dedent("""
        You are an expert in string manipulation tasks. You will be given a sequence of words and need to create a new string made from the last letter of each word. Before answering, explain your reasoning about how you'll extract and concatenate the letters.
          
        You will always respond with a JSON object matching the following JSON schema:
        """)
            + json.dumps(response_model.model_json_schema(), indent=2)
            + """\n\nFirst, provide your step by step reasoning in the "reasoning" field. Then, in the "answer" field, provide only the lowercase concatenated letters without any additional text."""
        )
    else:
        system_prompt = dedent("""
        You are an expert in string manipulation tasks. You will be given a sequence of words and need to create a new string made from the last letter of each word. Before answering, explain your reasoning about how you'll extract and concatenate the letters.
        
        You will always respond in the following format:
        
        <str, step by step reasoning about the answer>
        ANSWER: <str, lowercase concatenated last letters>
        
        First, provide your step by step reasoning. Then, in ANSWER, provide only the lowercase concatenated letters without any additional text.
        """)

    fewshot_examples = [
        (
            "Ian Peter Bernard Stephen",
            "The last letter of 'Ian' is 'n'. The last letter of 'Peter' is 'r'. The last letter of 'Bernard' is 'd'. The last letter of 'Stephen' is 'N'. Concatenating them is 'nrdn'.",
            "nrdn",
        ),
        (
            "Larry Page",
            "The last letter of 'Larry' is 'y'. The last letter of 'Page' is 'e'. Concatenating them is 'ye'.",
            "ye",
        ),
        (
            "Sergey Brin",
            "The last letter of 'Sergey' is 'y'. The last letter of 'Brin' is 'n'. Concatenating them is 'yn'.",
            "yn",
        ),
        (
            "Bill Gates",
            "The last letter of 'Bill' is 'l'. The last letter of 'Gates' is 's'. Concatenating them is 'ls'.",
            "ls",
        ),
        (
            "Jason Wei",
            "The last letter of 'Jason' is 'n'. The last letter of 'Wei' is 'i'. Concatenating them is 'ni'.",
            "ni",
        ),
        (
            "François Chollet",
            "The last letter of 'François' is 's'. The last letter of 'Chollet' is 't'. Concatenating them is 'st'.",
            "st",
        ),
        (
            "Yann LeCun",
            "The last letter of 'Yann' is 'n'. The last letter of 'LeCun' is 'n'. Concatenating them is 'nn'.",
            "nn",
        ),
        (
            "Eliezer Yudkowsky",
            "The last letter of 'Eliezer' is 'r'. The last letter of 'Yudkowsky' is 'y'. Concatenating them is 'ry'.",
            "ry",
        ),
    ]

    system_prompt += "\nExamples:\n"
    for i, (example_q, example_reason, example_ans) in enumerate(fewshot_examples):
        system_prompt += f"\n\n**{i+1}**\nQuestion: Take the last letters of the words in '{example_q}' and concatenate them."
        if prompt_type in [
            PromptType.WITH_JSON_MODE.value,
            PromptType.WITH_TOOL_CALLS.value,
            PromptType.WITH_STRICT_TOOL_CALLS.value,
        ]:
            response = f'{{"reasoning": "{example_reason}", "answer": "{example_ans}"}}'
        else:
            response = f"{example_reason}\nANSWER: {example_ans}"
        system_prompt += f"\nAssistant Response:\n{response}"

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"Question: {question}",
        },
    ]
    return messages


def parse_response_last_letter(response: str, prompt_type: str) -> str | None:
    if prompt_type in [
        PromptType.WITH_JSON_MODE.value,
        PromptType.WITH_TOOL_CALLS.value,
        PromptType.WITH_STRICT_TOOL_CALLS.value,
    ]:
        return ResponseLastLetter.model_validate_json(response).answer
    else:
        return response.split("\nANSWER:")[1].rstrip(".").strip()

In [271]:
dataset = load_dataset("ChilleD/LastLetterConcat")
evals = [
    {"question": d["question"], "answer": d["answer"].lower()} for d in dataset["test"]
][:10]

evaluator = LLMEvaluator(
    configs=CONFIGS,
    create_prompt_fn=create_prompt_last_letter,
    parse_response_fn=parse_response_last_letter,
    response_model=ResponseLastLetter,
)

df = evaluator.generate_outputs(evals)
df_results = evaluator.evaluate_outputs(df)

In [273]:
evaluator.calculate_confidence_intervals(df_results)
evaluator.run_paired_t_test(df_results)

Calculating confidence intervals (0.95) with 10 observations:
score_without_so - Mean: 80.00% CI: 53.87% - 106.13%
score_with_so_strict_tool_calls - Mean: 90.00% CI: 70.40% - 109.60%
score_with_so_tool_calls - Mean: 80.00% CI: 53.87% - 106.13%
score_with_so_json_mode - Mean: 90.00% CI: 70.40% - 109.60%

score_without_so vs score_with_so_tool_calls
t-statistic: 0.0, p-value: 1.0
score_without_so vs score_with_so_json_mode
t-statistic: -1.0, p-value: 0.34343639613791355
score_without_so vs score_with_so_strict_tool_calls
t-statistic: -1.0, p-value: 0.34343639613791355


## Shuffled objects   

In [276]:
class ResponseShuffledObjects(BaseModel):
    model_config = ConfigDict(extra="forbid", title="Response")
    reasoning: str = Field(description="step by step reasoning about the answer")
    answer: Literal["A", "B", "C", "D", "E"] = Field(description="final answer")


def create_prompt_shuffled_objects(
    question,
    prompt_type: str,
    response_model: ResponseShuffledObjects | None = None,
):
    if prompt_type in [
        PromptType.WITH_JSON_MODE.value,
        PromptType.WITH_TOOL_CALLS.value,
        PromptType.WITH_STRICT_TOOL_CALLS.value,
    ]:
        system_prompt = (
            dedent("""
        You are an expert in performing common sense tasks involving the ordering of a sequence of events.
        Each question will present you with a sequence of events and a question about it. Your task is to determine the correct answer from the options provided.
          
        You will always respond with a JSON object matching the following JSON schema:
        """)
            + json.dumps(response_model.model_json_schema(), indent=2)
            + dedent("""

        First, provide your step by step reasoning in the "reasoning" field. Then, in the "answer" field, provide only the single letter representing the correct choice you are presented with. Don't include any other text in the "answer" field.
        """)
        )
    else:
        system_prompt = dedent("""
        You are an expert in performing common sense tasks involving the ordering of a sequence of events.
        Each question will present you with a sequence of events and a question about it. Your task is to determine the correct answer from the options provided.
        
        You will always respond in the following format:
        
        <str, step by step reasoning about the answer>
        ANSWER: <str, final answer>
        
        First, provide your step by step reasoning. Then, in ANSWER, provide only the single letter representing the correct choice you are presented with. Don't include any other text in ANSWER.
        """)

    fewshot_examples = [
        (
            "Alice, Bob, Claire, Dave, and Eve are dancers at a square dance. At the start of a song, they each have a partner: Alice is dancing with Patrick, Bob is dancing with Sam, Claire is dancing with Jamie, Dave is dancing with Lola, and Eve is dancing with Melissa.\nThroughout the song, the dancers often trade partners. First, Dave and Eve switch partners. Then, Dave and Alice switch partners. Then, Eve and Alice switch partners. Then, Claire and Bob switch partners. Finally, Dave and Alice switch partners. At the end of the dance, Alice is dancing with\nOptions:\n(A) Patrick\n(B) Sam\n(C) Jamie\n(D) Lola\n(E) Melissa",
            "Dave and Eve switch partners, so Dave's partner is now Melissa and Eve's partner is now Patrick. Then Dave and Alice switch partners so Dave's partner is now Patrick and Alice's partner is now Melissa. Then Eve and Alice switch partners so Eve's partner is now Melissa and Alice's partner is now Lola. Then Claire and Bob switch patners so Claire's partner is now Sam, and Bob's partner is now Jamie. Finally, Dave and Alice switch partners so Dave's new partner is Lola, and Alice's new partner is Patrick. Alice is dance in with Patrick, choice A.",
            "A",
        ),
        (
            "Alice, Bob, Claire, Dave, and Eve are dancers at a square dance. At the start of a song, they each have a partner: Alice is dancing with Ophelia, Bob is dancing with Jamie, Claire is dancing with Melissa, Dave is dancing with Rodrigo, and Eve is dancing with Patrick.\nThroughout the song, the dancers often trade partners. First, Claire and Bob switch partners. Then, Claire and Eve switch partners. Then, Claire and Bob switch partners. Then, Eve and Dave switch partners. Finally, Claire and Alice switch partners. At the end of the dance, Alice is dancing with\nOptions:\n(A) Ophelia\n(B) Jamie\n(C) Melissa\n(D) Rodrigo\n(E) Patrick",
            "Claire and Bob switch partners, so Claire's partner is now Jamie and Bob's partner is now Melissa. Then, Claire and Eve switch partners, so Claire's partner becomes Patrick and Eve's partner becomes Jamie. Next, Claire and Bob switch partners again, making Claire's partner Melissa and Bob's partner Patrick. After that, Eve and Dave switch partners, resulting in Eve's partner being Rodrigo and Dave's partner being Jamie. Finally, Claire and Alice switch partners, so Claire's partner is now Ophelia and Alice's partner becomes Melissa. Alice is dancing with Melissa, which is choice C.",
            "C",
        ),
        (
            "Alice, Bob, Claire, Dave, and Eve are friends and avid readers who occasionally trade books. At the start of the semester, they each buy one new book: Alice gets Catch-22, Bob gets Hound of the Baskervilles, Claire gets Frankenstein, Dave gets The Pearl, and Eve gets The Fellowship of the Ring.\nAs the semester proceeds, they start trading around the new books. First, Eve and Alice swap books. Then, Alice and Claire swap books. Then, Alice and Bob swap books. Then, Dave and Alice swap books. Finally, Dave and Claire swap books. At the end of the semester, Dave has\nOptions:\n(A) Catch-22\n(B) Hound of the Baskervilles\n(C) Frankenstein\n(D) The Pearl\n(E) The Fellowship of the Ring",
            "Dave and Eve start by swapping their books, so Dave now has The Fellowship of the Ring and Eve has The Pearl. Next, Dave and Alice swap books, meaning Dave takes Alice's original book Catch-22 while Alice receives The Fellowship of the Ring. Then, Alice and Claire swap books, so Alice now holds Frankenstein and Claire gets The Fellowship of the Ring. After that, Alice and Bob swap books, resulting in Alice having Hound of the Baskervilles and Bob taking Frankenstein. Finally, Dave and Claire swap books, which means Dave ends up with The Fellowship of the Ring. Therefore, the correct answer is E.",
            "E",
        ),
    ]

    system_prompt += "\nExamples:\n"
    for i, (example_q, example_reason, example_ans) in enumerate(fewshot_examples):
        system_prompt += f"\n\n**{i+1}**\nQuestion: {example_q}"

        if prompt_type in [
            PromptType.WITH_JSON_MODE.value,
            PromptType.WITH_TOOL_CALLS.value,
            PromptType.WITH_STRICT_TOOL_CALLS.value,
        ]:
            response = f'{{"reasoning": "{example_reason}", "answer": "{example_ans}"}}'
        else:
            response = f"{example_reason}\nANSWER: {example_ans}"
        system_prompt += f"\nAssistant Response:\n{response}"

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"Question: {question}",
        },
    ]
    return messages


def parse_response_shuffled_objects(response: str, prompt_type: str) -> str:
    if prompt_type in [
        PromptType.WITH_JSON_MODE.value,
        PromptType.WITH_TOOL_CALLS.value,
        PromptType.WITH_STRICT_TOOL_CALLS.value,
    ]:
        return ResponseShuffledObjects.model_validate_json(response).answer
    else:
        return response.split("\nANSWER:")[1].rstrip(".").strip()

In [277]:
# | output: false
dataset = load_dataset(
    "openeval/BIG-Bench-Hard", data_files="tracking_shuffled_objects_five_objects.json"
)
evals = [
    {
        "question": d["input"],
        "answer": d["target"].replace("(", "").replace(")", "").strip(),
    }
    for d in dataset["train"]["examples"][0][4:]  # first 3 are few-shot examples
]

evaluator = LLMEvaluator(
    configs=CONFIGS,
    create_prompt_fn=create_prompt_shuffled_objects,
    parse_response_fn=parse_response_shuffled_objects,
    response_model=ResponseShuffledObjects,
)

df = evaluator.generate_outputs(evals)
df_results = evaluator.evaluate_outputs(df)

Repo card metadata block was not found. Setting CardData to empty.


In [278]:
evaluator.calculate_confidence_intervals(df_results)
evaluator.run_paired_t_test(df_results)

Calculating confidence intervals (0.95) with 10 observations:
score_without_so - Mean: 90.00% CI: 70.40% - 100.00%
score_with_so_strict_tool_calls - Mean: 80.00% CI: 53.87% - 100.00%
score_with_so_tool_calls - Mean: 70.00% CI: 40.06% - 99.94%
score_with_so_json_mode - Mean: 70.00% CI: 40.06% - 99.94%

score_without_so vs score_with_so_tool_calls
t-statistic: 1.4999999999999998, p-value: 0.16785065605707492
score_without_so vs score_with_so_json_mode
t-statistic: 1.0, p-value: 0.34343639613791355
score_without_so vs score_with_so_strict_tool_calls
t-statistic: 1.0, p-value: 0.34343639613791355


In [ ]:
# instructor trace: https://smith.langchain.com/public/7788f836-c63a-4dd1-9c4f-c7caf6fe93e5/r